In [1]:
import os
import cv2
import glob
from glob import glob
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from multiprocessing import Pool, cpu_count
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from IPython.display import display
import keras

Using TensorFlow backend.


In [2]:
def show(X):
    display(Image.fromarray(X))
    
def im_multi(path):
    try:
        #print(path)
        im_stats_im_ = cv2.imread(path)
        #print(im_stats_im_.size)
        return [path, {'size': im_stats_im_.size}]
    except:
        print("Error path:%s"%(path))
        return [path, {'size': [0, 0]}]
    
def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join([str(s) for s in str(im_stats_d[x]['size'])]))
    return im_stats_df

def get_im_cv2(args):
    img = cv2.imread(args[0])
    resized = cv2.resize(img, (args[1], args[1]), cv2.INTER_LINEAR)
    return [args[0], resized]

def image_features(paths, dim):
    imf_d = {}
    p = Pool(cpu_count())
    dims = [dim for i in range(len(paths.tolist()))]
    params = list(zip(paths.tolist(), dims))
    ret = p.map(get_im_cv2, params)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype = np.uint8)
    return fdata

In [3]:
all_cervix_images = []

basepath = '/home/dataset/train/'
for path in sorted(glob(basepath + "*")):
    #print(path)
    cervix_type = path.split("/")[-1]
    #print(cervix_type)
    cervix_images = sorted(glob(basepath + cervix_type + "/*"))
    #print(cervix_images)
    all_cervix_images = all_cervix_images + cervix_images

basepath = '/home/dataset/Type_'
for path in sorted(glob(basepath + "*")):
    #print(path)
    cervix_type = path.split("/")[-1][-1]
    #print(cervix_type)
    cervix_images = sorted(glob(basepath + cervix_type + "/*"))
    #print(cervix_images)
    all_cervix_images = all_cervix_images + cervix_images

In [4]:
all_cervix_images = pd.DataFrame({'imagepath': all_cervix_images})
all_cervix_images['filetype'] = all_cervix_images.apply(lambda row: row.imagepath.split(".")[-1], axis=1)
all_cervix_images['type'] = all_cervix_images.apply(lambda row: row.imagepath.split("/")[-2], axis=1)
all_cervix_images.head()

,imagepath,filetype,type
0,/home/dataset/train/Type_1/0.jpg,jpg,Type_1
1,/home/dataset/train/Type_1/10.jpg,jpg,Type_1
2,/home/dataset/train/Type_1/1013.jpg,jpg,Type_1
3,/home/dataset/train/Type_1/1014.jpg,jpg,Type_1
4,/home/dataset/train/Type_1/1019.jpg,jpg,Type_1


In [7]:
import glob
train1 = glob.glob('/home/dataset/train/**/*.jpg')
train1 =\
pd.DataFrame([[p.split('/')[4],p.split('/')[5],p] for p in train1], columns = ['type','image','path'])

train2 = glob.glob('/home/dataset/Type_*/*.jpg')
train2 =\
pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train2], columns = ['type','image','path'])

train = pd.concat([train1, train2]).reset_index(drop=True)
train.head()

,type,image,path
0,Type_3,477.jpg,/home/dataset/train/Type_3/477.jpg
1,Type_3,1153.jpg,/home/dataset/train/Type_3/1153.jpg
2,Type_3,335.jpg,/home/dataset/train/Type_3/335.jpg
3,Type_3,324.jpg,/home/dataset/train/Type_3/324.jpg
4,Type_3,98.jpg,/home/dataset/train/Type_3/98.jpg


In [6]:
test = glob.glob('/home/dataset/test/*.jpg')
test = pd.DataFrame([[p.split('/')[4],p] for p in test], columns = ['image','path'])

test_id = test.image.values
test.head()

,image,path
0,477.jpg,/home/dataset/test/477.jpg
1,178.jpg,/home/dataset/test/178.jpg
2,335.jpg,/home/dataset/test/335.jpg
3,324.jpg,/home/dataset/test/324.jpg
4,98.jpg,/home/dataset/test/98.jpg


In [10]:
test_id = test.image.values

In [13]:
train_ = im_stats(train)
test_ = im_stats(test)

Error path:/home/dataset/Type_1/5893.jpg
Error path:/home/dataset/Type_2/5892.jpg
Error path:/home/dataset/Type_2/2845.jpg


In [18]:
train_1 = train_[train_['size'] != '[ 0 ,   0 ]'].reset_index(drop=True) # remove bad images
test_1 = test_[test_['size'] != '[ 0 ,   0 ]'].reset_index(drop=True)    # remove bad images

In [19]:
train_1

,type,image,path,size
0,Type_3,477.jpg,/home/dataset/train/Type_3/477.jpg,2 3 9 7 0 8 1 6
1,Type_3,1153.jpg,/home/dataset/train/Type_3/1153.jpg,2 3 9 7 0 8 1 6
2,Type_3,335.jpg,/home/dataset/train/Type_3/335.jpg,2 3 9 7 0 8 1 6
3,Type_3,324.jpg,/home/dataset/train/Type_3/324.jpg,2 3 9 7 0 8 1 6
4,Type_3,98.jpg,/home/dataset/train/Type_3/98.jpg,2 8 7 5 5 6 4 8
5,Type_3,170.jpg,/home/dataset/train/Type_3/170.jpg,2 3 9 7 0 8 1 6
6,Type_3,1350.jpg,/home/dataset/train/Type_3/1350.jpg,2 3 9 7 0 8 1 6
7,Type_3,163.jpg,/home/dataset/train/Type_3/163.jpg,2 3 9 7 0 8 1 6
8,Type_3,344.jpg,/home/dataset/train/Type_3/344.jpg,2 3 9 7 0 8 1 6
9,Type_3,801.jpg,/home/dataset/train/Type_3/801.jpg,2 3 9 7 0 8 1 6


In [9]:
dim = 32
train_data = image_features(train_1['path'], dim)
test_data = image_features(test_1['path'], dim)

Process ForkPoolWorker-11:
Traceback (most recent call last):
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-3:
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-2779ab681428>

KeyboardInterrupt: 

Process ForkPoolWorker-9:
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-2-2779ab681428>", line 24, in get_im_cv2
    img = cv2.imread(args[0])
KeyboardInterrupt
Traceback (most recent call last):
Process ForkPoolWorker-12:
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/root/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/multiprocessi

In [ ]:
train_data.shape, test_data.shape

In [ ]:
train_1 = train_[train_['size'] != '[ 0 ,   0 ]'].reset_index(drop=True) #remove bad images
test_1 = test_[test_['size'] != '[ 0 ,   0 ]'].reset_index(drop=True)    #remove bad images